# Example Sample Hotel and Flight Booker Agent 

Dis solution go help you book flight ticket and hotel. Di scenario na trip wey go London Heathrow LHR Feb 20th 2024 go New York JFK, come back Feb 27th 2025, flying economy with British Airways only. I wan stay for Hilton hotel for New York, abeg provide di cost for di flight and hotel.


# Set up Azure AI Agent Service and get config info from **.env**

### **.env**

Make .env file

**.env** get di connection string for Azure AI Agent Service, di model wey AOAI dey use, and di Google API Search service API, ENDPOINT, etc.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Di name for your Azure AI Agent Service Model Deployment"

[**NOTE**] You go need model wey get 100,000 Rate Limit (Tokens per minute) and Rate Limit of 600 (Request per minute)

  You fit find di model for Azure AI Foundry - Model and Endpoint.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Di connection string for your Azure AI Agent Service Project"

  You fit find di project connection string for di project overview for AI Foundry Portal Screen.

- **SERPAPI_SEARCH_API_KEY** = "Di SERPAPI Search API KEY wey you get"
- **SERPAPI_SEARCH_ENDPOINT** = "Di SERPAPI Search Endpoint wey you get"

To find di Model Deployment Name and Project Connection String for Azure AI Agent Service, you go need create Azure AI Agent Service. E good make you use [dis template](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) to create am directly （***Note:*** Azure AI Agent Service dey available for limited region now. E good make you check [dis link](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) to set di region)

Agent go need access SERPAPI. E good make you register through [dis link](https://serpapi.com/searches). After you register, you go fit get unique API KEY and ENDPOINT.


# Setup

To run dis notebook, you go need make sure say you don install di required libraries by running `pip install -r requirements.txt`.


In [ ]:
from semantic_kernel import __version__

__version__

Your Semantic Kernel version suppose be at least 1.27.2.


Load your .env file setting and resources abeg make sure say you don add your keys and setting and don create local .env file.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Log in to Azure

You go need log in to Azure now. Open terminal and run dis command:

```bash
az login
```

Dis command go ask you to put your Azure login details, so dat Azure AI Agent service go fit work well.


# Explanation:
Dis na variable wey dey keep API key wey you go use take access SERP (Search Engine Results Page) API service. API key na special code wey dem dey use take confirm say na your account dey make request.

Purpose: Di reason for dis line na to keep di API key inside variable so e go fit dey use authenticate di requests wey you dey send go di SERP API service. Di API key na wetin you need to fit access di service and do search.

How to Get a SERP API Key: To get SERP API key, follow dis general steps for https://serpapi.com (di exact steps fit dey different based on di SERP API service wey you dey use):

Choose SERP API Service: Plenty SERP API services dey, like SerpAPI, Google Custom Search JSON API, and others. Pick di one wey go work well for wetin you need.

Sign Up for Account: Go di website for di SERP API service wey you choose and create account. You fit need provide small info and confirm your email address.

Create API Key: After you don sign up, log in to your account and go di API section or dashboard. Find di option wey go allow you create or generate new API key.
Copy di API Key go your .env file.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Explanation:
BASE_URL: Na one variable wey dey keep the base URL for the SERP API endpoint. The name BASE_URL na convention wey dem dey use to show say this URL na the starting point to take make API requests.
'https://serpapi.com/search':

This one na the actual URL string wey dem assign to the BASE_URL variable. E represent the endpoint wey dem dey use to do search queries with the SERP API.

# Purpose:
The reason for this line na to define one constant wey go hold the base URL for the SERP API. This URL go be the starting point to take build API requests wey go do search operations.

# Usage:
As dem define the base URL inside one variable, e go make am easy to reuse am for your code anytime you wan make requests to the SERP API. E go make your code easy to maintain and e go reduce the chance of mistake wey fit happen if you dey hardcode the URL for many places. The example wey dey now na https://serpapi.com/search?engine=bing wey dey use Bing search API. You fit choose different API for https://Serpapi.com


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Explanation:

Na here your plugin code dey.

Class Definition: `class BookingPlugin`: E dey define one class wey dem call BookingPlugin wey get methods to book hotels and flights.

Hotel Booking Method:

- `@kernel_function(description="booking hotel")`: Na one decorator wey dey describe di function as kernel function to book hotels.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: E dey define one method to book hotels wey get annotated parameters and return type.

Di method go build one dictionary of parameters for di hotel booking request and send GET request go di SERP API. E go check di response status and return di hotel properties if e work, or None if di request no work.

Flight Booking Method:

- `@kernel_function(description="booking flight")`: Na one decorator wey dey describe di function as kernel function to book flights.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: E dey define one method to book flights wey get annotated parameters and return type.

Di method go build dictionaries of parameters for di outbound and return flight requests and send GET requests go di SERP API. E go check di response status and add di flight information to di result string if e work, or e go print error message if di request no work. Di method go return di result string wey get di flight information.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Explanation:
Import Statements: Import di modules wey you go need for Azure credentials, AI agent, chat message content, author role, and kernel function decorator.

Asynchronous Context Manager: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): Dis one na to set up asynchronous context manager wey go handle Azure credentials and create AI agent client.

Agent Name and Instructions: 
- `AGENT_NAME = "BookingAgent"`: E dey define di name of di agent.
- `AGENT_INSTRUCTIONS = """..."""`: E dey give detailed instructions for di agent on how e go take handle booking requests.

Create Agent Definition: `agent_definition = await client.agents.create_agent(...)`: E dey create agent definition wey get di model, name, and instructions wey dem specify.

Create AzureAI Agent: `agent = AzureAIAgent(...)`: E dey create AzureAI agent wey dey use di client, agent definition, and di plugin wey dem define.

Create Thread: `thread: AzureAIAgentThread | None = None`: E dey create thread for di agent. E no dey compulsory to first create thread - if dem provide di value `None`, new thread go dey created during di first invocation and e go dey returned as part of di response.

User Inputs: `user_inputs = ["..."]`: E dey define list of user inputs wey di agent go process.

For di finally block, delete di thread and agent to clean up resources.


# Authentication

`DefaultAzureCredential` na one class wey dey inside Azure SDK for Python. E dey provide default way to take do authentication with Azure services. E go try authenticate using different methods for one specific order, like environment variables, managed identity, and Azure CLI credentials.

Asynchronous Operations: The aio module dey show say DefaultAzureCredential class fit support asynchronous operations. This one mean say you fit use am with asyncio to run authentication requests wey no go block.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Disclaimer**:  
Dis dokyument don use AI translet service [Co-op Translator](https://github.com/Azure/co-op-translator) do di translet. Even as we dey try make am correct, abeg make you sabi say AI translet fit get mistake or no dey accurate well. Di original dokyument wey dey for im native language na di one wey you go take as di correct source. For important informate, e good make you use professional human translet. We no go fit take blame for any misunderstanding or wrong interpretation wey fit happen because you use dis translet.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
